# 🗂️ SQLite SQL — A Comprehensive Guide for Learners and Developers

*A Deep Dive into the SQL Language of the SQLite Engine*



## 🧭 Introduction

- **SQLite** is a lightweight, server-less, self-contained SQL database engine that stores all its data in a single file. 

- It implements most of the SQL-92 standard and is used in everything from web browsers and mobile apps to embedded systems and operating-system utilities.

- Unlike server databases (SQL Server, MySQL, PostgreSQL), SQLite runs **in-process** with the application — there’s no external service to install or manage. 

- You simply open a file, execute SQL statements, and the engine handles data persistence, indexing, and transactions automatically.

- This guide provides a comprehensive overview of **SQLite’s SQL dialect**, its data model, and its unique implementation details. All examples use clean SQL syntax with explanatory commentary below each code block.

## ⚙️ Understanding SQLite Architecture

SQLite is a **file-based relational database**.

When you connect to a database such as `company.db`, SQLite creates (or opens) a single file on disk containing:

* Tables (data)

* Indexes

* Views

* Triggers

* The database schema

There’s no user authentication, no server configuration, and no networking — the database engine runs entirely in your application process.

SQLite supports both **persistent databases** (`.db` files) and **in-memory databases** (`:memory:`) that exist only during program execution.

## 🧱 Basic SQL Structure

The foundation of SQL in SQLite (and all relational systems) is built around four main statement categories:

| Category | Purpose | Examples |
|-----------|----------|-----------|
| **DDL (Data Definition Language)** | Create or alter database objects. | `CREATE TABLE`, `DROP TABLE`, `CREATE INDEX` |
| **DML (Data Manipulation Language)** | Insert, update, or delete data. | `INSERT`, `UPDATE`, `DELETE` |
| **DQL (Data Query Language)** | Retrieve data. | `SELECT`, `WITH`, `JOIN` |
| **DCL/Transaction Control** | Manage changes. | `BEGIN`, `COMMIT`, `ROLLBACK` |

Each section below explains these categories in practical detail.

## Syntax

- SQL is made up of commands. Commands are typically terminated by a semicolon, which marks the end 
of the command.

```sql
                select id, name from foods; 
                insert into foods values (null, 'Whataburger'); 
                delete from foods where id=413; 
```

- Commands, in turn, are composed of a series of tokens. 

- Tokens can be literals, keywords, identifiers, expressions, or special characters. 

- Tokens are separated by white space, such as spaces, tabs, and newlines.

- The semicolon is used by SQLite as a command terminator, signaling the end of the command to be processed. 

- Literals, also called constants, denote explicit values. There are three kinds: string literals, numeric 
literals, and binary literals. 

- String literals are one or more alphanumeric characters surrounded by single quotes.

```sql
                'Jerry' 
                'Newman' 
                'JujyFruit' 
```

- Keywords are words that have a specific meaning in SQL. These include words such as `SELECT`, `UPDATE`, `INSERT`, `CREATE`, `DROP`, and `BEGIN`. 

- Identifiers refer to specific objects within the database, such as tables or indexes. 

- Keywords are reserved words and may not be used as identifiers. 

- SQL is case insensitive with respect to keywords and identifiers. 

## ⚙️ Keywords

| **Keyword**           | **Description**                                                     | **Example**                                    |
| --------------------- | ------------------------------------------------------------------- | ---------------------------------------------- |
| **ABORT**             | Conflict resolution — aborts the command but leaves changes intact. | `INSERT OR ABORT INTO t VALUES(1);`            |
| **ACTION**            | Foreign key ON DELETE/UPDATE action.                                | `ON DELETE NO ACTION`                          |
| **ADD**               | ALTER TABLE: add a column.                                          | `ALTER TABLE t ADD COLUMN flag INT;`           |
| **AFTER**             | Trigger timing.                                                     | `CREATE TRIGGER tr AFTER INSERT ON t ...`      |
| **ALL**               | Return all rows, not DISTINCT.                                      | `SELECT ALL col FROM t;`                       |
| **ALTER**             | Modify schema objects.                                              | `ALTER TABLE t RENAME TO t2;`                  |
| **ANALYZE**           | Rebuild statistics for query planner.                               | `ANALYZE;`                                     |
| **AND**               | Logical conjunction.                                                | `WHERE a=1 AND b=2`                            |
| **AS**                | Alias specification.                                                | `SELECT a AS x FROM t;`                        |
| **ASC**               | Ascending sort order.                                               | `ORDER BY name ASC`                            |
| **ATTACH**            | Add a database file to the current connection.                      | `ATTACH 'other.db' AS other;`                  |
| **AUTOINCREMENT**     | Strong sequential INTEGER PRIMARY KEY behavior.                     | `id INTEGER PRIMARY KEY AUTOINCREMENT`         |
| **BEFORE**            | Trigger timing.                                                     | `CREATE TRIGGER tr BEFORE UPDATE ON t ...`     |
| **BEGIN**             | Begin a transaction.                                                | `BEGIN TRANSACTION;`                           |
| **BETWEEN**           | Range comparison.                                                   | `WHERE age BETWEEN 18 AND 30`                  |
| **BY**                | ORDER BY / GROUP BY.                                                | `GROUP BY category`                            |
| **CASCADE**           | Cascading FK updates/deletes.                                       | `ON DELETE CASCADE`                            |
| **CASE**              | Conditional expression.                                             | `CASE WHEN x>0 THEN 'pos' END`                 |
| **CAST**              | Type conversion.                                                    | `SELECT CAST(price AS INTEGER)`                |
| **CHECK**             | Constraint validation.                                              | `CHECK (qty > 0)`                              |
| **COLLATE**           | Apply a collation.                                                  | `ORDER BY name COLLATE NOCASE`                 |
| **COLUMN**            | Column context keyword.                                             | Used in `PRAGMA table_info(table)`             |
| **COMMIT**            | End a transaction and save changes.                                 | `COMMIT;`                                      |
| **CONFLICT**          | Conflict clause.                                                    | `ON CONFLICT REPLACE`                          |
| **CONSTRAINT**        | Name a constraint.                                                  | `CONSTRAINT pk PRIMARY KEY(id)`                |
| **CREATE**            | Create tables, indexes, triggers, or views.                         | `CREATE TABLE t (id INT);`                     |
| **CROSS**             | CROSS JOIN.                                                         | `SELECT * FROM a CROSS JOIN b;`                |
| **CURRENT**           | Used in datetime expressions.                                       | `CURRENT_TIMESTAMP`                            |
| **CURRENT_DATE**      | Built-in date literal.                                              | `SELECT CURRENT_DATE;`                         |
| **CURRENT_TIME**      | Built-in time literal.                                              | `SELECT CURRENT_TIME;`                         |
| **CURRENT_TIMESTAMP** | Built-in datetime literal.                                          | `SELECT CURRENT_TIMESTAMP;`                    |
| **DATABASE**          | Used in ATTACH/DETACH.                                              | `DETACH DATABASE other;`                       |
| **DEFAULT**           | Default column value.                                               | `value TEXT DEFAULT 'x'`                       |
| **DEFERRABLE**        | Delays FK checks.                                                   | `DEFERRABLE INITIALLY DEFERRED`                |
| **DEFERRED**          | Transaction/constraint mode.                                        | `BEGIN DEFERRED TRANSACTION;`                  |
| **DELETE**            | DML DELETE.                                                         | `DELETE FROM t WHERE id=1;`                    |
| **DESC**              | Descending sort order.                                              | `ORDER BY score DESC`                          |
| **DETACH**            | Remove attached DB.                                                 | `DETACH other;`                                |
| **DISTINCT**          | Remove duplicates.                                                  | `SELECT DISTINCT city FROM t;`                 |
| **DROP**              | Remove schema objects.                                              | `DROP TABLE t;`                                |
| **EACH**              | Trigger row-level indicator.                                        | `FOR EACH ROW`                                 |
| **ELSE**              | CASE alternative.                                                   | `CASE WHEN x>0 THEN 'p' ELSE 'n' END`          |
| **END**               | Terminates CASE blocks.                                             | `CASE … END`                                   |
| **ESCAPE**            | Escape character for LIKE.                                          | `LIKE '5\%' ESCAPE '\'`                        |
| **EXCEPT**            | Set subtraction.                                                    | `SELECT a FROM t1 EXCEPT SELECT a FROM t2;`    |
| **EXCLUSIVE**         | Transaction mode.                                                   | `BEGIN EXCLUSIVE;`                             |
| **EXISTS**            | Returns true if subquery has rows.                                  | `WHERE EXISTS (SELECT 1 FROM t)`               |
| **EXPLAIN**           | Show query plan.                                                    | `EXPLAIN QUERY PLAN SELECT * FROM t;`          |
| **FAIL**              | Conflict resolution — stops row but continues statement.            | `INSERT OR FAIL INTO t ...`                    |
| **FOR**               | FK clause keyword.                                                  | Used in `REFERENCES t(id) FOR EACH ROW`        |
| **FOREIGN**           | Foreign key.                                                        | `FOREIGN KEY (id) REFERENCES t(id)`            |
| **FROM**              | Select source table.                                                | `SELECT * FROM t`                              |
| **FULL**              | FULL JOIN (accepted syntax, not implemented).                       | `SELECT * FROM a FULL JOIN b`                  |
| **GLOB**              | Filename-style match.                                               | `WHERE name GLOB 'a*'`                         |
| **GROUP**             | GROUP BY.                                                           | `GROUP BY dept`                                |
| **HAVING**            | Group filter.                                                       | `HAVING COUNT(*) > 1`                          |
| **IF**                | Conditional CREATE/DROP.                                            | `CREATE TABLE IF NOT EXISTS t`                 |
| **IGNORE**            | Conflict resolution.                                                | `INSERT OR IGNORE INTO t ...`                  |
| **IMMEDIATE**         | Acquire reserved lock at BEGIN.                                     | `BEGIN IMMEDIATE;`                             |
| **IN**                | Set membership test.                                                | `WHERE id IN (1,2,3)`                          |
| **INDEX**             | Index creation.                                                     | `CREATE INDEX idx ON t(col);`                  |
| **INDEXED**           | Query optimization hint.                                            | `WHERE x=1 INDEXED BY idx`                     |
| **INITIALLY**         | FK deferral mode.                                                   | `INITIALLY DEFERRED`                           |
| **INNER**             | INNER JOIN.                                                         | `SELECT * FROM a INNER JOIN b`                 |
| **INSERT**            | DML INSERT.                                                         | `INSERT INTO t VALUES (1,2)`                   |
| **INSTEAD**           | INSTEAD OF TRIGGER.                                                 | `CREATE TRIGGER tr INSTEAD OF INSERT ON v ...` |
| **INTERSECT**         | Set intersection.                                                   | `SELECT a FROM t1 INTERSECT SELECT a FROM t2;` |
| **INTO**              | INSERT target.                                                      | `INSERT INTO t(col) VALUES (...);`             |
| **IS**                | Comparison operator.                                                | `WHERE x IS NULL`                              |
| **ISNULL**            | Equivalent to `IS NULL`.                                            | `x ISNULL`                                     |
| **JOIN**              | Joining tables.                                                     | `SELECT * FROM a JOIN b ON a.id=b.id`          |
| **KEY**               | NOT strictly used by SQLite except FK docs.                         | Seen in `PRIMARY KEY`                          |
| **LEFT**              | LEFT JOIN.                                                          | `LEFT JOIN t2 ON ...`                          |
| **LIKE**              | Pattern matching.                                                   | `WHERE name LIKE 'A%'`                         |
| **LIMIT**             | Row limit.                                                          | `SELECT * FROM t LIMIT 10;`                    |
| **MATCH**             | FTS or FK match operator.                                           | `MATCH 'query'`                                |
| **NATURAL**           | NATURAL JOIN.                                                       | `SELECT * FROM a NATURAL JOIN b`               |
| **NO**                | FK ON NO ACTION.                                                    | `ON UPDATE NO ACTION`                          |
| **NOT**               | Logical NOT.                                                        | `WHERE NOT (x=1)`                              |
| **NOTNULL**           | Equivalent to `NOT NULL`.                                           | `col TEXT NOTNULL`                             |
| **NULL**              | Null literal.                                                       | `INSERT INTO t VALUES(NULL)`                   |
| **OF**                | Trigger target columns.                                             | `UPDATE OF column1 ON t`                       |
| **OFFSET**            | Skip N rows.                                                        | `LIMIT 10 OFFSET 20`                           |
| **ON**                | JOIN / TRIGGER / FK.                                                | `ON DELETE CASCADE`                            |
| **OR**                | Logical OR.                                                         | `WHERE a=1 OR b=1`                             |
| **ORDER**             | ORDER BY.                                                           | `ORDER BY name`                                |
| **OUTER**             | OUTER JOIN.                                                         | `LEFT OUTER JOIN table2`                       |
| **PLAN**              | EXPLAIN QUERY PLAN.                                                 | `EXPLAIN QUERY PLAN SELECT …`                  |
| **PRAGMA**            | SQLite-specific operations.                                         | `PRAGMA table_info(t);`                        |
| **PRIMARY**           | Primary key.                                                        | `PRIMARY KEY(id)`                              |
| **QUERY**             | EXPLAIN QUERY PLAN.                                                 | `EXPLAIN QUERY PLAN SELECT …`                  |
| **RAISE**             | Trigger exception.                                                  | `SELECT RAISE(ABORT, 'msg')`                   |
| **RECURSIVE**         | CTE recursion.                                                      | `WITH RECURSIVE ...`                           |
| **REFERENCES**        | FK reference.                                                       | `REFERENCES Parent(id)`                        |
| **REGEXP**            | Regex operator (requires extension).                                | `col REGEXP '^[A-Z]'`                          |
| **REINDEX**           | Rebuild index.                                                      | `REINDEX idx;`                                 |
| **RELEASE**           | Release savepoint.                                                  | `RELEASE savept1;`                             |
| **RENAME**            | ALTER TABLE rename.                                                 | `ALTER TABLE t RENAME TO new_t;`               |
| **REPLACE**           | Conflict resolution.                                                | `INSERT OR REPLACE INTO t ...`                 |
| **RESTRICT**          | FK restriction.                                                     | `ON DELETE RESTRICT`                           |
| **RIGHT**             | RIGHT JOIN (accepted only).                                         | `RIGHT JOIN t2`                                |
| **ROLLBACK**          | Undo transaction.                                                   | `ROLLBACK;`                                    |
| **ROW**               | Row-value expressions.                                              | `SELECT ROW(1,2,3);`                           |
| **SAVEPOINT**         | Nested transaction point.                                           | `SAVEPOINT s1;`                                |
| **SELECT**            | Query rows.                                                         | `SELECT * FROM t;`                             |
| **SET**               | UPDATE clause.                                                      | `UPDATE t SET col=1`                           |
| **TABLE**             | Table declaration.                                                  | `CREATE TABLE t (...);`                        |
| **TEMP**              | Temporary table.                                                    | `CREATE TEMP TABLE t(...);`                    |
| **TEMPORARY**         | Synonym for TEMP.                                                   | `CREATE TEMPORARY TABLE t`                     |
| **THEN**              | CASE THEN clause.                                                   | `CASE WHEN x THEN y END`                       |
| **TO**                | ALTER TABLE rename.                                                 | `RENAME TO newname`                            |
| **TRANSACTION**       | Transaction control.                                                | `BEGIN TRANSACTION;`                           |
| **TRIGGER**           | Create trigger.                                                     | `CREATE TRIGGER tgr ...`                       |
| **UNION**             | Set union.                                                          | `SELECT a FROM t1 UNION SELECT a FROM t2`      |
| **UNIQUE**            | Constraint or SELECT modifier.                                      | `UNIQUE (email)`                               |
| **UPDATE**            | Modify rows.                                                        | `UPDATE t SET col='x'`                         |
| **USING**             | Join condition.                                                     | `JOIN t2 USING (id)`                           |
| **VACUUM**            | Rebuild database file physically.                                   | `VACUUM;`                                      |
| **VALUES**            | INSERT block.                                                       | `INSERT INTO t VALUES (1,'x')`                 |
| **VIEW**              | Create view.                                                        | `CREATE VIEW v AS SELECT ...`                  |
| **VIRTUAL**           | Virtual table.                                                      | `CREATE VIRTUAL TABLE t USING fts5`            |
| **WHEN**              | CASE WHEN clause.                                                   | `CASE WHEN x=1 THEN 'yes' END`                 |
| **WHERE**             | Filter clause.                                                      | `WHERE id>10`                                  |
| **WITH**              | Start CTE.                                                          | `WITH cte AS (...) SELECT ...`                 |
| **WITHOUT**           | WITHOUT ROWID optimization.                                         | `CREATE TABLE t(...) WITHOUT ROWID`            |


## 🏗️ String Functions

| Function                  | Description                             | Example                                 |
| ------------------------- | --------------------------------------- | --------------------------------------- |
| **LENGTH(x)**             | Returns character length of string *x*. | `SELECT LENGTH('abc'); -- 3`            |
| **LOWER(x)**              | Converts to lowercase.                  | `SELECT LOWER('ABC');`                  |
| **UPPER(x)**              | Converts to uppercase.                  | `SELECT UPPER('abc');`                  |
| **SUBSTR(x, start, len)** | Extract substring (1-based index).      | `SELECT SUBSTR('abcdef', 2, 3); -- bcd` |
| **TRIM(x)**               | Removes leading/trailing whitespace.    | `SELECT TRIM('  hi  ');`                |
| **LTRIM(x)**              | Trim left side.                         | `SELECT LTRIM('  hi');`                 |
| **RTRIM(x)**              | Trim right side.                        | `SELECT RTRIM('hi  ');`                 |
| **REPLACE(x,y,z)**        | Replace substring *y* with *z*.         | `SELECT REPLACE('abc', 'b', 'X');`      |
| **INSTR(x, y)**           | Index of substring *y* in *x*.          | `SELECT INSTR('abcd','c'); -- 3`        |
| **PRINTF(fmt, …)**        | C-style string formatting.              | `SELECT PRINTF('%.2f', 1.2345);`        |
| **QUOTE(x)**              | Returns SQL literal form of string.     | `SELECT QUOTE('a''b'); -- 'a''b'`       |
| **UNICODE(x)**            | Unicode code point of 1st char.         | `SELECT UNICODE('A'); -- 65`            |
| **CHAR(n, …)**            | Convert integers to characters.         | `SELECT CHAR(65,66,67); -- ABC`         |
| **HEX(x)**                | Binary → hex string.                    | `SELECT HEX('AB'); -- 4142`             |


## 💻 Math Functions

| Function          | Description                 | Example                           |
| ----------------- | --------------------------- | --------------------------------- |
| **ABS(x)**        | Absolute value.             | `SELECT ABS(-5);`                 |
| **ROUND(x, n)**   | Rounded to *n* digits.      | `SELECT ROUND(1.234, 2); -- 1.23` |
| **RANDOM()**      | Random int (64-bit).        | `SELECT RANDOM();`                |
| **RANDOMBLOB(n)** | N-byte random blob.         | `SELECT RANDOMBLOB(16);`          |
| **SIGN(x)**       | -1, 0, or 1 depending on x. | `SELECT SIGN(-20); -- -1`         |
| **CEIL(x)**       | Smallest integer ≥ x.       | `SELECT CEIL(1.1);`               |
| **FLOOR(x)**      | Largest integer ≤ x.        | `SELECT FLOOR(1.9);`              |


## 📦 Date/Time Functions

| Function              | Description                  | Example                               |
| --------------------- | ---------------------------- | ------------------------------------- |
| **DATE(x, …)**        | Returns date.                | `SELECT DATE('now');`                 |
| **TIME(x, …)**        | Returns time.                | `SELECT TIME('now');`                 |
| **DATETIME(x, …)**    | Combined date + time.        | `SELECT DATETIME('now','localtime');` |
| **JULIANDAY(x)**      | Days since noon in 4714 B.C. | `SELECT JULIANDAY('2020-01-01');`     |
| **STRFTIME(fmt, x)**  | Custom formatter.            | `SELECT STRFTIME('%Y-%m-%d', 'now');` |
| **CURRENT_DATE**      | Today’s date.                | `SELECT CURRENT_DATE;`                |
| **CURRENT_TIME**      | Time now.                    | `SELECT CURRENT_TIME;`                |
| **CURRENT_TIMESTAMP** | Date/time now.               | `SELECT CURRENT_TIMESTAMP;`           |


## 📊 Aggregate Functions

| Function                 | Description                     | Example                                  |
| ------------------------ | ------------------------------- | ---------------------------------------- |
| **COUNT(x)**             | Count non-NULL rows.            | `SELECT COUNT(*) FROM t;`                |
| **SUM(x)**               | Sum numeric column.             | `SELECT SUM(price) FROM t;`              |
| **AVG(x)**               | Average value.                  | `SELECT AVG(score) FROM t;`              |
| **MIN(x)**               | Minimum value.                  | `SELECT MIN(age) FROM people;`           |
| **MAX(x)**               | Maximum value.                  | `SELECT MAX(age) FROM people;`           |
| **TOTAL(x)**             | Sum as floating point (always). | `SELECT TOTAL(price);`                   |
| **GROUP_CONCAT(x, sep)** | Concatenate strings in group.   | `SELECT GROUP_CONCAT(name, ',') FROM t;` |


## 🖼️  Window Functions

| Function            | Description                 | Example                             |
| ------------------- | --------------------------- | ----------------------------------- |
| **ROW_NUMBER()**    | Sequential numbering.       | `ROW_NUMBER() OVER (ORDER BY id)`   |
| **RANK()**          | Rank with gaps.             | `RANK() OVER (ORDER BY score DESC)` |
| **DENSE_RANK()**    | Rank without gaps.          | `DENSE_RANK() OVER (...)`           |
| **PERCENT_RANK()**  | Percent-based rank.         | `PERCENT_RANK() OVER (...)`         |
| **CUME_DIST()**     | Cumulative distribution.    | `CUME_DIST() OVER (...)`            |
| **NTILE(n)**        | Break rows into n buckets.  | `NTILE(4) OVER (ORDER BY score)`    |
| **LAG(x, n, d)**    | Access prior row.           | `LAG(score) OVER (ORDER BY id)`     |
| **LEAD(x, n, d)**   | Access following row.       | `LEAD(score) OVER (ORDER BY id)`    |
| **FIRST_VALUE(x)**  | First row’s value in frame. | `FIRST_VALUE(name) OVER (...)`      |
| **LAST_VALUE(x)**   | Last row in window frame.   | `LAST_VALUE(name) OVER (...)`       |
| **NTH_VALUE(x, n)** | nth row in frame.           | `NTH_VALUE(salary,2) OVER (...)`    |


## ✨ JSON Functions

| Function                       | Description                    | Example                             |
| ------------------------------ | ------------------------------ | ----------------------------------- |
| **JSON(value)**                | Convert SQL value → JSON.      | `SELECT JSON('[1,2]');`             |
| **JSON_ARRAY(v1, …)**          | Build JSON array.              | `SELECT JSON_ARRAY(1,2,3);`         |
| **JSON_OBJECT(k,v, …)**        | Build JSON object.             | `SELECT JSON_OBJECT('a',1);`        |
| **JSON_INSERT(j, path, val)**  | Insert without replacing.      | `JSON_INSERT(j,'$.x',5)`            |
| **JSON_REPLACE(j, path, val)** | Replace existing values.       | `JSON_REPLACE(j,'$.x',5)`           |
| **JSON_SET(j, path, val)**     | Insert or replace.             | `JSON_SET(j,'$.x',5)`               |
| **JSON_REMOVE(j, path)**       | Remove key/path.               | `JSON_REMOVE(j,'$.a')`              |
| **JSON_EXTRACT(j,path …)**     | Extract JSON values.           | `JSON_EXTRACT(j,'$.a')`             |
| **->**                         | Shorthand extract operator.    | `json -> '$.a'`                     |
| **->>**                        | Extract as text.               | `json ->> '$.a'`                    |
| **JSON_TYPE(j,path)**          | Returns type (string, array…). | `JSON_TYPE(j,'$.a')`                |
| **JSON_VALID(j)**              | Check well-formed JSON.        | `SELECT JSON_VALID('x');`           |
| **JSON_QUOTE(x)**              | Escape as JSON string.         | `JSON_QUOTE('A"B')`                 |
| **JSON_PATCH(j1,j2)**          | RFC-7396 patch merge.          | `JSON_PATCH('{"a":1}','{"a":2}')`   |
| **JSON_GROUP_ARRAY(x)**        | Aggregate → JSON array.        | `SELECT JSON_GROUP_ARRAY(name)`     |
| **JSON_GROUP_OBJECT(k,v)**     | Aggregate → JSON object.       | `SELECT JSON_GROUP_OBJECT(id,name)` |


## 🤖 Miscellaneous Functions

| Function                | Description                                      | Example                       |
| ----------------------- | ------------------------------------------------ | ----------------------------- |
| **CHANGES()**           | Number of rows modified by last statement.       | `SELECT CHANGES();`           |
| **TOTAL_CHANGES()**     | Total modified in session.                       | `SELECT TOTAL_CHANGES();`     |
| **LAST_INSERT_ROWID()** | Last rowid in session.                           | `SELECT LAST_INSERT_ROWID();` |
| **ZEROBLOB(n)**         | Returns a zero-filled BLOB of length n.          | `SELECT ZEROBLOB(100);`       |
| **TYPEOF(x)**           | Returns one of: text, integer, real, blob, null. | `SELECT TYPEOF(5.2);`         |
| **COALESCE(x1,x2,…)**   | First non-NULL argument.                         | `SELECT COALESCE(NULL,5);`    |
| **IFNULL(x,y)**         | x if not null, else y.                           | `SELECT IFNULL(NULL, 10);`    |
| **NULLIF(x,y)**         | NULL if x=y else x.                              | `SELECT NULLIF(3,3);`         |


## 🧠 NULL

- Most relational databases support the concept of “unknown” or “unknowable” through a special placeholder called `NULL`, which is a placeholder for missing information and is not a value per se. 

- `NULL` is the absence of a value. 

- `NULL` is not nothing. 

- `NULL` is not something. 

- `NULL` is not true. 

- `NULL` is not false. 

- `NULL` is not zero. 

- `NULL` is not an empty string. 

## 📚 CREATE Command

- Tables store data as rows and columns, similar to spreadsheets but strongly typed and indexed.


```
        CREATE [TEMP] TABLE table_name (column_definitions [, constraints]); 

```

In [ ]:
CREATE TABLE Employees 
(
    EmployeeID   INTEGER PRIMARY KEY,
    FirstName    TEXT NOT NULL,
    LastName     TEXT NOT NULL,
    Department   TEXT,
    HireDate     TEXT DEFAULT CURRENT_DATE,
    Salary       REAL
);

**Explanation:**

* `INTEGER PRIMARY KEY` defines a unique identifier. In SQLite, this automatically creates an **alias for the internal rowid** (a 64-bit integer unique to each record).

* `TEXT`, `REAL`, and `INTEGER` are **type affinities**, not rigid types — SQLite stores values dynamically while maintaining data consistency.

* `DEFAULT CURRENT_DATE` automatically inserts the current date on record creation.

### Viewing Tables

In [ ]:
.tables

In [ ]:
.schema Employees

### Temporary Tables

```

        CREATE [TEMP|TEMPORARY] TABLE table_name (column_definitions [, constraints]);

```

- The temp or temporary keyword creates a temporary table that will last only as long your session. 

## 🧠 ALTER Command

- You can change parts of a table with the alter table command. SQLite’s version of alter table can 
either rename a table or add columns. 

- The general form of the command is as follows: 

```

        ALTER TABLE table_name { rename to name | add column column_def } 

```



In [ ]:
ALTER TABLE contacts 
ADD column email TEXT NOT NULL DEFAULT '' COLLATE nocase;

## 🔍 Querying Data

- The select command incorporates many of the relational operations through a series of clauses in its syntax. 

- Each clause corresponds to specific relational operation. 

In [ ]:
SELECT [distinct] heading 
FROM tables 
WHERE condition
GROUP BY columns 
HAVING condition 
ORDER BY columns 
LIMIT count, offset; 

- Each keyword — `FROM`, `WHERE`, `HAVING`, and so on—is a separate clause. 

- Each clause consists of the keyword followed by arguments 

- The `SELECT` command starts with `FROM`, which takes one or more input relations; combines them into a single composite relation; and passes it through the subsequent chain of operations. 

- All clauses are optional with the exception of `SELECT`. You must always provide at least this clause to make a valid `SELECT` command.

```sql

        SELECT heading FROM tables WHERE condition;

``` 

- The `FROM` clause is a comma-separated list of one or more tables

## ✍️ INSERT Command

- Syntax

In [ ]:
INSERT INTO table_name (column_list) 
VALUES (value_list); 

- Usage

In [ ]:
INSERT INTO Employees (FirstName, LastName, Department, Salary)
VALUES ('Jane', 'Doe', 'Finance', 85000);

In [ ]:
INSERT INTO Employees (FirstName, LastName, Department, Salary)
VALUES
  ('John', 'Smith', 'HR', 72000),
  ('Alice', 'Brown', 'IT', 95000),
  ('Bob', 'Miller', 'Finance', 78000);

- SQLite enforces constraints (`NOT NULL`, `UNIQUE`, `PRIMARY KEY`) automatically.

- Subqueries can be used in insert statements, both as components of the value list and as a complete 
replacement of the value list. 

- When you specify a subquery as the value list, you are really inserting a set of rows, because you are inserting the set of rows returned by that subquery. 





In [ ]:
CREATE TABLE list AS 
SELECT f.name food, t.name name, 
    (SELECT count(episode_id) 
    FROM foods_episodes WHERE food_id=f.id) episodes 
FROM foods f, food_types t 
WHERE f.type_id=t.id; 

SELECT * 
FROM list; 

## 🔍 SELECT Statement

In [ ]:
SELECT FirstName, LastName, Department
FROM Employees
WHERE Department = 'Finance'
ORDER BY LastName;

In [ ]:
SELECT FirstName || ' ' || LastName AS FullName, Salary AS AnnualPay
FROM Employees;

## ⚖️ WHERE Clause

Use logical and comparison operators to refine results.

| Operator | Meaning | Example |
|-----------|----------|----------|
| `=` | Equal to | `WHERE Department = 'IT'` |
| `<>` or `!=` | Not equal | `WHERE Department <> 'Finance'` |
| `<`, `>`, `<=`, `>=` | Comparison | `WHERE Salary >= 90000` |
| `AND`, `OR`, `NOT` | Combine conditions | `WHERE Department='IT' AND Salary>80000` |

In [ ]:
SELECT * 
FROM Employees 
WHERE Department IS NULL;

## 🧠 Execution Order

- SQL queries are not executed in the same order they’re written. 

- SQLite follows a logical processing order that determines how clauses are evaluated.

```

        FROM → WHERE → GROUP BY → HAVING → SELECT → ORDER BY → LIMIT

```

- SQL applies the `WHERE` clause to each row of the relation produced by the `FROM` clause

- The argument of `WHERE` is a logical predicate. A predicate, in the simplest sense, is just an assertion about something.

- The subject in the `WHERE` clause is a row. The row is the logical subject. The `WHERE` clause is the logical 
predicate. 

- Each row in which the proposition evaluates to true is included (or selected) as part of the 
result. 

- Each row in which it is false is excluded. 

## 🧰 Logical Operators

- Logical operators (`AND`, `OR`, `NOT`, `IN`) are binary operators that operate on truth values or logical 
expressions. 

- They produce a specific truth value depending on their inputs.

- A useful relational operator is `LIKE`. 

- `LIKE` is similar to equals (=) but is used for matching string values against patterns. 

- A percent symbol (`%`) in the pattern matches any sequence of zero or more characters in the string. 

- An underscore (`_`) in the pattern matches any single character in the string. 

## 🔢 Sorting and Limiting Results

- Limit the size and particular range of the result using the `LIMIT` and `OFFSET` keywords. 

- `LIMIT` specifies the maximum number of records to return. 

- `OFFSET` specifies the number of records to skip. 

-  `ORDER BY`. This clause sorts the result by a column or columns before it is returned. 

- This is important in this example is because the rows returned from `SELECT` are never guaranteed to be in any specific order—the SQL standard declares this. 

- The `ORDER BY` clause is essential if you need to count on the result being in any specific order. 

In [ ]:
SELECT * FROM Employees
ORDER BY Salary DESC;

In [ ]:
SELECT * FROM Employees
ORDER BY EmployeeID
LIMIT 5 OFFSET 10;

## 🛠️ CASE Expression

- The `CASE` expression allows you to handle various conditions within a `SELECT` statement. 

- There are two forms. 

- The first and simplest form takes a static value and lists various case values linked to return 
values:

#### Syntax

In [ ]:
CASE value 
    WHEN x THEN value_x 
    WHEN y THEN value_y 
    WHEN z THEN value_z 
    ELSE default_value 
END 

#### Usage

In [ ]:
SELECT name || CASE type_id 
                    WHEN 7  THEN ' is a drink' 
                    WHEN 8  THEN ' is a fruit' 
                    WHEN 9  THEN ' is junkfood' 
                    WHEN 13 THEN ' is seafood' 
                    ELSE NULL 
                END description 

- The `CASE` expression in the above example handles a few different `type_id values`, returning a string 
appropriate for each one. 

- The returned value is called `description`, as qualified after the `END` keyword. 

- This string is concatenated to name by the string concatenation operator (`||`),

- For all `type_ids` not specified in a `WHERE` condition, `CASE` returns null. 

- The `SELECT` statement filters out such null values in the `WHERE` clause, so all that is returned are rows that the `CASE` expression does handle. 

- The second form of `CASE` allows for expressions in the `WHEN` condition. 

#### Syntax

In [ ]:
CASE
    WHEN condition1 THEN value1 
    WHEN condition2 THEN value2 
    WHEN condition3 THEN value3 
    ELSE default_value 
END

#### Usage

In [ ]:
SELECT name,(SELECT 
               CASE 
                    WHEN count(*) > 4 THEN 'Very High' 
                    WHEN count(*) = 4 THEN 'High' 
                    WHEN count(*) IN (2,3) THEN 'Moderate' 
                    ELSE 'Low' 
               END 
             FROM foods_episodes 
             WHERE food_id=f.id) frequency 
FROM foods f 
WHERE frequency LIKE '%High' 

## 🔗 Joining Tables

- Joins are the key to working with data from multiple tables (or relations) and are the first operation(s) of 
the select command. 

- The result of a join is provided as the input or starting point for all subsequent 
(filtering) operations in the select command. 

- Joins in SQLite are best understood by example. 

- Ex. The `foods` table has a column `type_id`. The values in this column correspond to values in the id column in the `food_types` table. 


In [ ]:
SELECT foods.name, food_types.name 
FROM foods, food_types 
WHERE foods.type_id=food_types.id LIMIT 10; 


- A relationship exists between the two tables. Any value in the `foods.type_id` column must correspond to a 
value in the `food_types.id` column, and the `id` column is the `primary key` (described later) of food_types. 

- The `foods.type_id` column, by virtue of this relationship, is called a `foreign key`: it contains (or 
references) values in the primary key of another table. 

- This relationship is called a `foreign key relationship `

- In practice, when you are joining tables, it is always a good idea to use the `table_name.column_name` notation to avoid any possible ambiguity. 

- To carry out the join, the database finds these matched rows. 

- For each row in the first table, the database finds all rows in the second table that have the same value for the joined columns and includes them in the input relation. 

- So in this example, the `FROM` clause built a composite relation by joining the rows of two tables

In [ ]:
SELECT e.FirstName, e.LastName, d.DepartmentName
FROM Employees AS e
INNER JOIN Departments AS d
ON e.Department = d.DepartmentName;

In [ ]:
SELECT c.CustomerName, o.OrderID
FROM Customers AS c
LEFT JOIN Orders AS o
ON c.CustomerID = o.CustomerID;

### INNER JOIN


- An inner join uses another set operation in relational algebra, called an intersection, to find elements that exist in both sets.

- Given the matching columns, an inner join finds the rows from both sides that contain like values and combines them to form the rows of the result 

- Inner joins only return rows that satisfy the given column relationship, also called the `join condition`. 

In [ ]:
SELECT * 
FROM foods 
INNER JOIN food_types 
ON foods.id = food_types.id ;

### CROSS JOIN

- The Cartesian join is one of the fundamental relational operations. 

- It is a brute-force join that results in the combination of all rows from the first table with all rows in the second. 

- `FROM`, in the absence of anything else, produces a c`ross join`. 

- In a cross join, no relationship exists between the rows; there is no join condition, but they are simply 
jammed together. 

In [ ]:
SELECT * 
FROM foods, food_types;

### OUTER JOIN

- An outer join selects all the rows of an inner join plus some rows outside of the relationship. 

- The three outer join types are called `LEFT`, `RIGHT`, and `FULL`.  

- A `LEFT OUTER JOIN` operates with respect to the “left table” in the SQL command. 

In [ ]:
SELECT * 
FROM foods 
LEFT OUTER JOIN foods_episodes 
ON foods.id=foods_episodes.food_id; 

- The `LEFT OUTER JOIN `tries to match every row of `foods` with every row in `foods_episodes` per the join 
condition (`foods.id`=`foods_episodes.food_id`): all such matching rows are included in the result.

- The remaining food rows of `foods` that don’t match `foods_episodes` are still included in the 
result, and where `foods_episodes` hasn’t provide a row, it supplies `NULL` results. 

- The `RIGHT OUTER JOIN` works similarly, except the right table is the one whose rows are included, 
matching or not. 

- Operationally, `LEFT OUTER JOIN` and `RIGHT OUTER JOIN` are identical; they do the same thing. They 
differ only in order and syntax. 

- Any problem that can requires a `RIGHT OUTER JOIN` for solution can equally be solved with a `LEFT OUTER JOIN`. 

- A `FULL OUTER JOIN` is the combination of a `LEFT OUTER JOIN` and `RIGHT OUTER JOIN`: it includes all matching records, followed by unmatched records in the right and left tables. 

- Currently, both `RIGHT OUTER JOIN` and `FULL OUTER JOIN` are **NOT** supported in SQLite. 

- However, a `RIGHT OUTER JOIN` can be replaced with a `LEFT OUTER JOIN`, and a `FULL OUTER JOIN` can be performed using compound queries.

### NATURAL JOIN

- A NATURAL JOIN joins two tables by their common column names. 

- Using the NATURAL JOIN, you can get the INNER JOIN of two tables without having to add the join condition. 

- Natural joins will join all columns by the same name in both tables. 

- The process of adding to or removing a column from a table can drastically change the results of a natural join query.

- It’s always better to explicitly define the join conditions of your queries than rely on the semantics of the table schema

In [ ]:
SELECT * 
FROM foods, food_types 
WHERE foods.id=food_types.food_id; 

- When the database sees more than one table listed, it knows there will be a join—at the very least a 
`CROSS JOIN`. 

- The where clause above calls for an `INNER JOIN`.

### Preferred Syntax

In [ ]:
SELECT heading 
FROM left_table join_type right_table 
ON join_condition; 

SELECT * 
FROM foods 
INNER JOIN food_types 
ON foods.id=food_types.food_id; 


SELECT * 
FROM foods 
LEFT OUTER JOIN food_types 
ON foods.id=food_types.food_id; 

SELECT * 
FROM foods 
CROSS JOIN food_types; 

## 🎭 Aliasing

- Syntax

In [ ]:
SELECT base-name AS alias

- Usage

In [ ]:
SELECT foods.name, food_types.name 
FROM foods, food_types 
WHERE foods.type_id = food_types.id 
LIMIT 10; 

SELECT f.name, t.name 
FROM foods f, food_types t 
WHERE f.type_id = t.id 
LIMIT 10; 

## 🧮 GROUP BY Statement

- In addition to computing aggregates over an entire result, you can also split that result into groups of rows with like values and compute aggregates on each group—all in one step. 

- `GROUP BY` sits in between the `WHERE` clause and the `SELECT` clause. 

- `GROUP BY` takes the output of where and splits it into groups of rows that share a common value (or values) for a specific column (or columns). 

- When `GROUP BY` is used, the select clause applies aggregates to each group separately, rather than the entire result as a whole. 

- Since aggregates produce a single value from a group of values, they collapse these groups of rows into single rows. 

In [ ]:
SELECT Department, AVG(Salary) AS AvgSalary
FROM Employees
GROUP BY Department
HAVING AVG(Salary) > 80000;

## 📦 HAVING Clause

-  The purpose of `HAVING`, a predicate that you apply to the result of `GROUP BY`, filters the groups from `GROUP BY` in the 
same way that the `WHERE` clause filters rows from the from clause. 

- The difference is that the `WHERE` clause’s predicate is expressed in terms of individual row values, and `HAVING`’s predicate is expressed in terms of aggregate values. 

In [ ]:
SELECT type_id, count(*) 
FROM foods 
GROUP BY type_id 
HAVING count(*) < 20; 

- `GROUP BY` and `HAVING` work as additional restriction phases. 

- `GROUP BY` takes the restriction produced by the `WHERE` clause and breaks it into groups of rows that share a common value for a given column. 

- `HAVING` then applies a filter to each of these groups. 

- The groups that make it through are passed on to the `SELECT` clause for aggregation and projection. 

## 🧩 Subqueries

- Subqueries are `SELECT` statements within `SELECT` statements. They are also called **subselects**. 

In [ ]:
SELECT FirstName, LastName
FROM Employees
WHERE Department IN (
    SELECT DepartmentName FROM Departments WHERE Location = 'HQ'
);

In [ ]:
SELECT c.CustomerName
FROM Customers AS c
WHERE EXISTS (
    SELECT 1 FROM Orders AS o WHERE o.CustomerID = c.CustomerID
);

- The most common use of subqueries is in the `WHERE` clause, specifically using the `IN`
operator. 

- The `IN` operator is a binary operator that takes an input value and a list of values and returns 
true if the input value exists in the list, or false otherwise. 


In [ ]:
SELECT name,  
    (SELECT COUNT(id) FROM foods_episodes WHERE food_id=f.id ) count 
FROM foods f 
ORDER BY count DESC 
LIMIT 10;


- Subqueries in the `SELECT` clause can be used to add additional data from other tables to the result 
set.

- Subqueries can be used in the `ORDER BY` clause as well.

In [ ]:
SELECT * 
FROM foods f 
ORDER BY (SELECT count(type_id) 
    FROM foods 
    WHERE type_id=f.type_id) DESC; 

SELECT f.name, types.name 
FROM foods f 
INNER JOIN (SELECT * 
            FROM food_types 
            WHERE id=6) types 
ON f.type_id=types.id; 

- Notice that the use of a subquery in the `FROM` clause requires a rename operation. In this case, the 
subquery was named types. 

- Subqueries as a source relation in the `FROM` clause are often called **inline views** or **derived tables**. 

- The thing to remember about subqueries is that they can be used anywhere a relational expression can be used.

## ⚙️ Updating and Deleting Data

In [ ]:
UPDATE Employees
SET Salary = Salary * 1.10
WHERE Department = 'Sales';

In [ ]:
DELETE FROM Employees
WHERE HireDate < '2020-01-01';

## 🧱 Indexes and Keys

In [ ]:
CREATE INDEX idx_employees_department
ON Employees (Department);

In [ ]:
DROP INDEX idx_employees_department;

### Enabling Foreign Keys

In [ ]:
PRAGMA foreign_keys = ON;

## 🧾 Views

In [ ]:
CREATE VIEW vw_HighEarners AS
SELECT FirstName, LastName, Department, Salary
FROM Employees
WHERE Salary > 90000;

In [ ]:
SELECT * FROM vw_HighEarners;

In [ ]:
DROP VIEW vw_HighEarners;

## 🔄 Transactions

In [ ]:
BEGIN TRANSACTION;
UPDATE Accounts SET Balance = Balance - 500 WHERE AccountID = 1;
UPDATE Accounts SET Balance = Balance + 500 WHERE AccountID = 2;
COMMIT;

## 🧩 Triggers

In [ ]:
CREATE TRIGGER trg_UpdateAudit
AFTER UPDATE ON Employees
FOR EACH ROW
BEGIN
    INSERT INTO AuditLog (Action, TableName, RecordID, Timestamp)
    VALUES ('UPDATE', 'Employees', OLD.EmployeeID, datetime('now'));
END;

In [ ]:
DROP TRIGGER trg_UpdateAudit;

## 🧮 The WITH Clause (CTEs)

In [ ]:
WITH DeptAverage AS (
    SELECT Department, AVG(Salary) AS AvgSalary
    FROM Employees
    GROUP BY Department
)
SELECT e.FirstName, e.LastName, e.Salary, d.AvgSalary
FROM Employees AS e
JOIN DeptAverage AS d ON e.Department = d.Department
WHERE e.Salary > d.AvgSalary;

## 📊 The PRAGMA Command

In [ ]:
PRAGMA foreign_keys = ON;
PRAGMA table_info(Employees);
PRAGMA database_list;
PRAGMA encoding;

## ⚡ Backup, Export, and Import

In [ ]:
.mode csv
.output employees.csv
SELECT * FROM Employees;
.output stdout

In [ ]:
.mode csv
.import employees.csv Employees

## ⚠️ Common Pitfalls and Best Practices

| Issue | Recommendation |
|--------|----------------|
| **Dynamic Typing** | Use `CHECK` constraints for strict validation. |
| **Foreign Keys Off by Default** | Always enable `PRAGMA foreign_keys=ON;`. |
| **Date Handling** | Dates stored as text; use `strftime()`. |
| **Transactions** | Use explicit transactions for performance. |
| **NULLs in Aggregates** | Use `COALESCE()` to replace NULLs. |
| **Case Sensitivity** | Use `COLLATE BINARY` for strict matching. |

## 🧩 Summary

SQLite’s SQL implementation is both **compact and powerful**.

Key takeaways:

* SQLite is **serverless** — everything lives in one file.

* It uses **dynamic typing** and **type affinities**.

* Supports **transactions, joins, triggers, and views**.

* Ideal for local storage and embedded systems.